In [9]:
from datasets import load_dataset

dataset = load_dataset(
    "/Users/huangxinzhe/code/llm_fine_tuning/hugging_face_transformers/fine_tuning/fine_tune_quickstart/datasets/yelp_review_full")

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "/Volumes/WD_BLACK/models/bert-base-cased")

# 以最大长度进行padding，并截断


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [11]:
len(tokenized_datasets['train'][0]['input_ids'])

124

In [12]:
small_train_dataset = tokenized_datasets["train"].shuffle(
    seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(
    seed=42).select(range(1000))

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/Users/huangxinzhe/code/llm_fine_tuning/hugging_face_transformers/fine_tuning/fine_tune_quickstart/models/bert-base-cased",
                                                           num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /Users/huangxinzhe/code/llm_fine_tuning/hugging_face_transformers/fine_tuning/fine_tune_quickstart/models/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import TrainingArguments

# 训练超参数
model_dir = "models/bert-base-cased"
# 训练过程指标监控
# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [15]:
import evaluate
import numpy as np
# 训练过程中的指标评估
metric = evaluate.load("./accuracy/")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [16]:
from transformers import Trainer

trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=small_train_dataset,
                  eval_dataset=small_eval_dataset,
                  compute_metrics=compute_metrics,
                  )

In [17]:
trainer.train()

ValueError: expected sequence of length 121 at dim 1 (got 139)